In [1]:
from numpy import *
U,Sigma,VT = linalg.svd([[1,1],[7,7]])

In [2]:
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [3]:
Sigma

array([1.00000000e+01, 2.82797782e-16])

In [4]:
VT

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

In [5]:
def loadExData():
    return[[1,1,1,0,0],
          [2,2,2,0,0],
          [1,1,1,0,0],
          [5,5,5,0,0],
          [1,1,0,2,2],
          [0,0,0,3,3],
          [0,0,0,1,1]]

In [6]:
Data=loadExData()

In [7]:
U,Sigma,VT=linalg.svd(Data)

In [8]:
Sigma

array([9.72140007e+00, 5.29397912e+00, 6.84226362e-01, 4.11502614e-16,
       1.36030206e-16])

In [9]:
Sig3 = mat([[Sigma[0],0,0],[0,Sigma[1],0],[0,0,Sigma[2]]])

In [10]:
Sig3

matrix([[9.72140007, 0.        , 0.        ],
        [0.        , 5.29397912, 0.        ],
        [0.        , 0.        , 0.68422636]])

In [11]:
U[:,:3]*Sig3*VT[:3,:]

matrix([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          7.75989921e-16,  7.71587483e-16],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          3.00514919e-16,  2.77832253e-16],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          2.18975112e-16,  2.07633779e-16],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
          3.00675663e-17, -1.28697294e-17],
        [ 1.00000000e+00,  1.00000000e+00, -5.48397422e-16,
          2.00000000e+00,  2.00000000e+00],
        [ 3.21319929e-16,  4.43562065e-16, -3.48967188e-16,
          3.00000000e+00,  3.00000000e+00],
        [ 9.71445147e-17,  1.45716772e-16, -1.52655666e-16,
          1.00000000e+00,  1.00000000e+00]])

In [12]:
from numpy import linalg as la

In [13]:
def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))

In [14]:
def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA,inB,rowvar=0)[0][1]

In [15]:
def cosSim(inA,inB):
    num = floate(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [16]:
myMat=mat(loadExData())

In [17]:
ecludSim(myMat[:,0],myMat[:,4])

0.13367660240019172

In [19]:
ecludSim(myMat[:,0],myMat[:,0])

1.0

In [21]:
def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item].A>0, dataMat[:j].A>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item],dataMat[overLap,j])
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [22]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]